In [1]:
using LinearAlgebra
using EmpiricalOrthogonalFunctions
using JLD2
using GLMakie

In [12]:
include("eof.jl")

eof

In [3]:
include("deprecated_functions.jl")

calculate_eofs_of_ensemble (generic function with 1 method)

In [4]:
include("utils.jl")

sample_along_line (generic function with 1 method)

In [5]:
# data_base_path = "/home/denis/workspace/data/ma_data"
data_base_path = "/mnt/bigdrive/Datasets/master_thesis_data"

"/mnt/bigdrive/Datasets/master_thesis_data"

In [6]:
ps_data_monthly_path = joinpath(data_base_path, "ps_data_monthly")
ivt_data_monthly_path = joinpath(data_base_path, "ivt_fields_monthly")
eof_data_path = joinpath(data_base_path, "eof_data")

"/mnt/bigdrive/Datasets/master_thesis_data/eof_data"

In [7]:
function filter_winter_season(time_element)
    winter_months = [12, 1, 2, 3]
    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

filter_winter_season (generic function with 1 method)

In [8]:
(ivt_historical_monthly, ivt_ssp126_monthly, ivt_ssp585_monthly) = build_ensemble_data(ivt_data_monthly_path, "historical", "ssp126", "ssp585"; file_range_selection=:, data_field_id="ivt", member_range=1:50, filterfun = filter_winter_season)

Handling scenario historical ...
Handling scenario ssp126 ...
Handling scenario ssp585 ...


3-element Vector{EnsembleSimulation}:
 EnsembleSimulation("historical", Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("1850-01-16T12:00:00"), DateTime("1850-02-14T21:00:00"), DateTime("1850-03-16T09:00:00"), DateTime("1850-12-16T09:00:00"), DateTime("1851-01-16T09:00:00"), DateTime("1851-02-14T21:00:00"), DateTime("1851-03-16T09:00:00"), DateTime("1851-12-16T09:00:00"), DateTime("1852-01-16T09:00:00"), Date

In [9]:
all_scopes = get_sliding_time_scopes_by_threshold(ivt_ssp585_monthly.time, 50)

37-element Vector{UnitRange{Int64}}:
 1:199
 4:203
 8:207
 12:211
 16:215
 20:219
 24:223
 28:227
 32:231
 36:235
 ⋮
 112:311
 116:315
 120:319
 124:323
 128:327
 132:331
 136:335
 140:339
 144:343

In [8]:
data = rand(20, 30, 100) * 1000

20×30×100 Array{Float64, 3}:
[:, :, 1] =
  19.4028  727.993   312.819   836.964   …  326.947   206.875   409.711
  25.7436  545.223   411.374    86.623      497.95    453.771    67.2075
 658.21    957.411   288.685   216.541       86.9203  233.297   267.232
 409.432    89.1184  826.089   249.137      609.227   225.559   786.407
 463.489   235.061   291.903   888.377      399.827   647.627   984.771
 829.904   815.469   430.939    26.4233  …  629.083   244.44    783.467
 844.959   476.152   690.624    51.8701     415.801   583.732   627.896
 673.757   684.842   128.906   674.637      193.564   811.296   572.581
 759.455   353.488   218.398    53.1464     164.378   539.157   405.572
 774.83    429.751   804.027   964.21       707.726   881.956   688.519
 464.866   914.031   584.761    69.8969  …  889.69    242.531   648.155
 151.004   533.19    871.685   724.259      834.85    293.19    923.624
 547.245   448.738    20.3921   29.9961     654.685   410.479    90.4942
  21.9553   11.1867  

In [11]:
weights = sqrt.(cos.(deg2rad.(51:80)))

30-element Vector{Float64}:
 0.7932971644029981
 0.7846409849897329
 0.7757673769578406
 0.7666715413346664
 0.7573482926309705
 0.7477920188600216
 0.7379966361813766
 0.7279555372639217
 0.7176615322769182
 0.7071067811865476
 ⋮
 0.5558929702514211
 0.5407140692849935
 0.5250117673128852
 0.5087426118407232
 0.49185556375796713
 0.4742900529674483
 0.45597334441583254
 0.4368168899854319
 0.41671114415975297

In [7]:
mean_pre = mean(data, dims=3)

weighted_data = data .* reshape(weights, 1, :, 1)

mean_after = weighted_data .- mean(weighted_data, dims=3)

extrema(abs.((data .- mean(data, dims=3)) .* reshape(weights, 1, :, 1) .- mean_after))

(0.0, 5.115907697472721e-13)

In [10]:
eof_response = eof(data; weights=nothing, timedim=3, weightdim=2, center=true, nmodes=-1, norm_withsqrt_timedim=false, align_eofs_with_mean=false)
eof_markert = get_eof_of_datachunk(data; center=true)
#eof_response.spatial_modes
# size(eof_response.spatial_modes)
extrema(abs.(eof_response.spatial_modes[:, :, 1:99] .- eof_markert.spatial_modes[: ,:, 1:99]))

(0.0, 2.984279490192421e-13)

In [13]:
realign_modes(eof_response, [ones(20 * 30) for _ in 1:100])

EOFResult([0.031015095713551318 0.024457571483741614 … 0.019274516501365718 0.08204431751035944; -0.04279688794892253 0.031651845146323104 … -0.01040469627978513 -0.06898666303298301; … ; 0.04489949956290775 0.009871340825062053 … -0.009227517007148921 0.005733827335821068; 0.06489155611247098 -0.039237824290539625 … 0.008208147370833212 0.022059624530798072;;; -0.048224588905862405 -0.013605646713172232 … -0.021301623777553444 -0.033489491557673196; -0.03363285030984487 0.026933564916744104 … -0.0019231114062962403 -0.10212857165664176; … ; -0.008580536937223288 0.06872890002430886 … 0.03998499471060198 -0.027018136983649063; 0.0024623877642139143 -0.015008973146123716 … 0.041270784343230206 0.01817892081130104;;; 0.01898392848514147 -0.04345087630998794 … -0.073700421895415 0.000911417044581546; -0.020162629011024433 0.0018224648128873306 … 0.08387003981465 -0.006346601218797585; … ; 0.0024618237452060975 0.005807286685868445 … -0.02254447174257583 -0.0007761835519348991; -0.11390327

In [53]:
surface(abs.(eof_response.spatial_modes .- eof_markert.spatial_modes)[:, :, 1])

In [22]:
# compare with python eof

weighted_eof_response = eof(data; weights=weights, timedim=3, weightdim=2, center=true, nmodes=-1, norm_withsqrt_timedim=false, align_eofs_with_mean=false)

poeof_result = pyeof_of_datachunk(data, 100; weights=weights)

diffs = weighted_eof_response.spatial_modes[:, :, 1:99] .- poeof_result.spatial_modes[:, :, 1:99]

extrema(abs.(diffs))

(1.4261027003196226e-7, 0.15783433490275298)

In [19]:
eof_test_reconstruction = eof(data; weights=weights, timedim=3, weightdim=2, center=true, nmodes=-1, norm_withsqrt_timedim=true, align_eofs_with_mean=true)

EOFResult([0.0034424513197309175 0.06642521203340462 … -0.05689794327925144 0.048911176707684446; 0.10413791526626973 0.1247302543373115 … 0.011596173283630442 0.0594156353900761; … ; 0.04396819917556998 0.05893765312493828 … 0.025233016266850268 -0.009222369939772152; -0.057908225650233625 -0.005627092791464103 … -0.07276020502793086 0.0016717752492261158;;; -0.0810609638032933 -0.04032384120641487 … -0.017506068770341974 0.07980263825746138; -0.026539383859163355 0.028917536962169997 … 0.020796242966518445 -0.01352416319595406; … ; -0.03963132602532757 -0.06811655819328773 … -0.027840049715815322 -0.03793719479927067; -0.07025720279207526 -0.03216035209904129 … 0.09856900971753596 -0.04374195374582664;;; 0.09744584912765593 0.0615147933985298 … -0.016474102378616184 -0.05389723708772011; 0.04307495998074127 -0.011443647419290761 … 0.0471887661616666 -0.03447021418698112; … ; 0.05972253066080135 -0.09966487469378536 … 0.008724693694917865 0.03881565265263037; 0.04707543304048588 0.054

In [46]:
norm(weighted_eof_response.temporal_modes[ :, 1])

1.0000000000000004

In [20]:
reconstructed_data = reconstruct_data(eof_test_reconstruction)

20×30×100 Array{Float64, 3}:
[:, :, 1] =
 -309.698    342.781      86.2718  …  -434.952   -121.286   -354.7
  406.419    120.277       1.841      -292.905    205.438    437.158
  107.138     13.9945   -279.814       402.122   -178.45    -279.884
  221.772     46.6674    -60.4964      -47.4752  -407.131    -67.3364
  425.658    351.902     340.717       451.434    153.655     52.6013
 -503.601   -291.526    -424.731   …   463.095   -290.307    359.831
 -380.274   -270.223     -26.7723     -253.178   -260.959    358.172
  -32.9741  -282.936     358.506       320.614    -12.3411   305.036
  195.732    -62.8635    397.877      -201.935   -158.703   -170.038
  -87.4217    -5.87866   -65.9514      419.452    -84.417   -478.757
 -108.624   -392.831     350.957   …  -495.88    -407.939    407.61
 -420.37    -393.609    -118.071       146.989   -172.535   -100.811
  135.349     82.8899   -299.176       304.554   -199.103     46.7494
 -361.006   -286.871     140.198       106.544    295.453    -

In [21]:
reconstructed_diffs = (data .- mean(data, dims=3)) .- reconstructed_data

extrema(abs.(reconstructed_diffs))

(0.0, 3.637978807091713e-12)

In [23]:
result = calculate_eofs_of_ensemble_fast(
    ivt_ssp585_monthly,
    all_scopes,
    5;
    center=true,
    align_eofs_with_mean=true,
    norm_withsqrt_timedim=false,
    geoweights=true,
    scale_mode=noscaling,
    saving_filepath=nothing,
    multithreading = false
)

0.0%┣                                               ┫ 0/50 [00:00<00:00, -0s/it]
ssp585 r1i1p1f1 2.0%┣▌                          ┫ 1/50 [00:13<Inf:Inf, InfGs/it]
ssp585 r2i1p1f1 4.0%┣█▎                             ┫ 2/50 [00:26<20:45, 26s/it]
ssp585 r3i1p1f1 6.0%┣█▉                             ┫ 3/50 [00:38<14:52, 19s/it]
ssp585 r4i1p1f1 8.0%┣██▌                            ┫ 4/50 [00:50<12:47, 17s/it]


InterruptException: InterruptException: